In [2]:
import cv2
import os

total_no_of_images = len(os.listdir('images'))
count = 0
for item, file in enumerate(os.listdir('images')):
    src = cv2.imread(cv2.samples.findFile('images/'+file)) if 'raw' in file else None
    if src is None:
        continue
    blurred = cv2.GaussianBlur(src, (11, 11), 0)
    width, height = src.shape[:2]
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, (10, 100, 100), (20, 255, 255))
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
                            cv2.CHAIN_APPROX_SIMPLE)
    print(cnts)
    # cnts = imutils.grab_contours(cnts)\n",
    new_cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    center = None
    if len(new_cnts) > 0:
        count += 1
        c = max(new_cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]),
        # To see the centroid clearly\n",
        if radius > 10 and radius < 65:
            cv2.circle(src, center, int(radius), (0, 255, 0), 2)
            cv2.imwrite(f"contour_detection_results/{file}", src)
            cv2.circle(src, center, 1, (0, 0, 255), 2)
    # print(center)\n",
    cv2.imshow('result', src)
    cv2.waitKey(500)
    cv2.destroyAllWindows()
    break
print(count/(total_no_of_images//2))

((array([[[784, 506]],

       [[783, 507]],

       [[778, 507]],

       [[777, 508]],

       [[773, 508]],

       [[772, 509]],

       [[769, 509]],

       [[768, 510]],

       [[766, 510]],

       [[765, 511]],

       [[754, 511]],

       [[753, 512]],

       [[752, 511]],

       [[742, 511]],

       [[741, 512]],

       [[738, 512]],

       [[737, 513]],

       [[736, 513]],

       [[734, 515]],

       [[734, 516]],

       [[733, 517]],

       [[733, 526]],

       [[734, 527]],

       [[736, 527]],

       [[737, 528]],

       [[739, 528]],

       [[741, 530]],

       [[741, 533]],

       [[742, 534]],

       [[742, 536]],

       [[743, 536]],

       [[744, 537]],

       [[745, 537]],

       [[746, 538]],

       [[747, 538]],

       [[748, 539]],

       [[749, 539]],

       [[753, 543]],

       [[754, 543]],

       [[755, 544]],

       [[756, 544]],

       [[757, 545]],

       [[757, 549]],

       [[758, 550]],

       [[771, 550]],

       [